Load the war/abort witch captions dataset

In [29]:
import pandas as pd
df = pd.read_csv("abortion_war_captions.csv")
df.head()

,Unnamed: 0,experiments,total_experiments,total_samples,interface,countries,languages,search_queries,video_id,video_title,...,creator_url,creator_link,user_action_sets,origin_revs,samples,trials,begins,ends,average_order,captions
0,1018,{'general'},0,0,tk0,{'nl'},{'en'},{'anti-war candidate european elections 2024 n...,7304639473243933984,A far-right populist who has previously called...,...,https://www.tiktok.com/@vicenews,<a href='https://www.tiktok.com/@vicenews' tar...,{'general_adversarial5__nl__en__netherlands'},{'2024-04-28T15:30:00'},{'3906'},{'eu24--tk0_scheduled__2024-04-30T06:40:00+00:...,{'2024-04-30T08:14:17.574597'},{'2024-04-30T08:14:17.574632'},6.0,An anti Islam politician who wants to ban mo...
1,1019,{'general'},0,0,tk0,{'nl'},{'en'},{'anti-war candidate european elections 2024 n...,7304691326711385377,New Dutch PM wants Islam banned #itvnews #neth...,...,https://www.tiktok.com/@itvnews,<a href='https://www.tiktok.com/@itvnews' targ...,{'general_adversarial5__nl__en__netherlands'},{'2024-04-28T15:30:00'},{'3906'},{'eu24--tk0_scheduled__2024-04-30T06:40:00+00:...,{'2024-04-30T08:14:17.574597'},{'2024-04-30T08:14:17.574632'},3.0,This man wants a ban on the Quran and to sh...
2,1020,{'general'},0,0,tk0,{'nl'},{'en'},{'anti-war candidate european elections 2024 n...,7304737421478874400,The anti-Islam populist nicknamed Mozart has j...,...,https://www.tiktok.com/@bbcnews,<a href='https://www.tiktok.com/@bbcnews' targ...,{'general_adversarial5__nl__en__netherlands'},{'2024-04-28T15:30:00'},{'3906'},{'eu24--tk0_scheduled__2024-04-30T06:40:00+00:...,{'2024-04-30T08:14:17.574597'},{'2024-04-30T08:14:17.574632'},8.0,"Meet Harris Builders, the guy who just won ..."
3,1021,{'general'},0,0,tk0,{'nl'},{'en'},{'anti-war candidate european elections 2024 n...,7304785915040320801,Geerts Wilders’ far-right Freedom Party has wo...,...,https://www.tiktok.com/@criticalanalysisuk,<a href='https://www.tiktok.com/@criticalanaly...,{'general_adversarial5__nl__en__netherlands'},{'2024-04-28T15:30:00'},{'3906'},{'eu24--tk0_scheduled__2024-04-30T06:40:00+00:...,{'2024-04-30T08:14:17.574597'},{'2024-04-30T08:14:17.574632'},0.0,Gert Wilder's far right anti Islam party has...
4,1022,{'general'},0,0,tk0,{'nl'},{'en'},{'anti-war candidate european elections 2024 n...,7305171428117155104,What do you think of this? Let's treat each ot...,...,https://www.tiktok.com/@tv.dutch,<a href='https://www.tiktok.com/@tv.dutch' tar...,{'general_adversarial5__nl__en__netherlands'},{'2024-04-28T15:30:00'},{'3906'},{'eu24--tk0_scheduled__2024-04-30T06:40:00+00:...,{'2024-04-30T08:14:17.574597'},{'2024-04-30T08:14:17.574632'},2.0,Today I have a message for the Turks. Your ...


Installing VADER lexicon

In [30]:
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk

# Download VADER if not already installed
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/adri/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

Applying VADER

In [31]:
# Initialize the VADER sentiment analyzer
sia = SentimentIntensityAnalyzer()

# Score from -1 to 1, where -1 is the most negative and 1 the most positive
df['sentiment_score'] = df['captions'].astype(str).apply(lambda text: sia.polarity_scores(text)['compound'])

In [32]:
print(df[['captions', 'sentiment_score']].head())

                                            captions  sentiment_score
0    An anti Islam politician who wants to ban mo...           0.8445
1    This man wants a ban on the Quran  and to sh...          -0.7040
2    Meet Harris Builders,  the guy who just won ...           0.9939
3    Gert Wilder's far right anti Islam party has...          -0.9813
4    Today I have a message for the Turks.  Your ...          -0.4104


Translation of the scores into a label

In [33]:
def categorize_sentiment(score):
    if score <= -0.75:
        return "Very Negative"
    elif score <= -0.4:
        return "Negative"
    elif score < 0:
        return "Slightly Negative"
    elif score == 0:
        return "Neutral"
    elif score < 0.4:
        return "Slightly Positive"
    elif score < 0.75:
        return "Positive"
    else:
        return "Very Positive"

In [34]:
df['sentiment_category'] = df['sentiment_score'].apply(categorize_sentiment)


Getting subjectivity of the content

In [35]:
from textblob import TextBlob

# Function to get subjectivity
def get_subjectivity(text):
    return TextBlob(text).sentiment.subjectivity

In [36]:
df['subjectivity'] = df['captions'].astype(str).apply(get_subjectivity)

In [37]:
print(df[['captions', 'sentiment_score', 'subjectivity']].head())

                                            captions  sentiment_score  \
0    An anti Islam politician who wants to ban mo...           0.8445   
1    This man wants a ban on the Quran  and to sh...          -0.7040   
2    Meet Harris Builders,  the guy who just won ...           0.9939   
3    Gert Wilder's far right anti Islam party has...          -0.9813   
4    Today I have a message for the Turks.  Your ...          -0.4104   

   subjectivity  
0      0.398934  
1      0.379031  
2      0.454266  
3      0.522713  
4      0.408333  
